In [1]:
import pandas as pd

In [2]:
# import preprocessed data from before
df = pd.read_csv("https://raw.githubusercontent.com/fwznbg/gemas/main/Indo_hatespeech/dfHatespeechPreprocessed.csv?token=ANRCLKBSOWW7YXMFKSL54W3BMG2X2")

In [12]:
# remove unnamed column
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Tweet'] = df['Tweet'].astype('str')

In [13]:
df.head()

,Tweet,HS
0,cowok usaha lacak perhati gue lantas remeh per...,1
1,telat tau edan sarap gue gaul cigax jifla cal ...,0
2,41 kadang pikir percaya tuhan jatuh kali kali ...,0
3,ku tau mata sipit lihat,0
4,kaum cebong kafir lihat dongok dungu haha,1


In [7]:
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# tokening

In [14]:
# split train and test first, only tokenize from the train test
X = df['Tweet']
y = df['HS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=99)

In [15]:
# USED CONSTANTS
MAX_WORDS = 5000 #berapa banyak yg diketahui sama si tokenizer
MAX_ENTRY_LEN = 150 #tiap entry berapa banyak, feeling aja sih dari statistik panjang di atas

In [16]:
# CREATE TOKENIZER OBJECTS
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='OOV')
tokenizer.fit_on_texts(X_train)

#printing properties, print word_index if you want lol
word_index = tokenizer.word_index

In [18]:
# making the padded sequences
sequences = tokenizer.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=MAX_ENTRY_LEN)

In [39]:
# imports
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [40]:
embedding_dim = 100

model = Sequential()
model.add(Embedding(MAX_WORDS, embedding_dim, input_length=MAX_ENTRY_LEN))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = Adam(learning_rate = 0.0001)
model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 150, 100)          500000    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 146, 128)          64128     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 565,429
Trainable params: 565,429
Non-trainable params: 0
_________________________________________________________________


In [41]:
history = model.fit(sequences_matrix, y_train,
                    epochs=10,
                    validation_split=0.1,
                    batch_size=10)

Epoch 1/10
949/949 [==============================] - 22s 23ms/step - loss: 0.6429 - accuracy: 0.6261 - val_loss: 0.5651 - val_accuracy: 0.7476
Epoch 2/10
949/949 [==============================] - 21s 22ms/step - loss: 0.4299 - accuracy: 0.8178 - val_loss: 0.3907 - val_accuracy: 0.8207
Epoch 3/10
949/949 [==============================] - 21s 23ms/step - loss: 0.3111 - accuracy: 0.8700 - val_loss: 0.3644 - val_accuracy: 0.8387
Epoch 4/10
949/949 [==============================] - 22s 23ms/step - loss: 0.2575 - accuracy: 0.8940 - val_loss: 0.3704 - val_accuracy: 0.8349
Epoch 5/10
949/949 [==============================] - 22s 23ms/step - loss: 0.2193 - accuracy: 0.9142 - val_loss: 0.3861 - val_accuracy: 0.8321
Epoch 6/10
949/949 [==============================] - 22s 23ms/step - loss: 0.1884 - accuracy: 0.9312 - val_loss: 0.4110 - val_accuracy: 0.8397
Epoch 7/10
949/949 [==============================] - 21s 23ms/step - loss: 0.1617 - accuracy: 0.9424 - val_loss: 0.4410 - val_accuracy: